# Benchmarking / Profiling

In [5]:
%load_ext autoreload
%autoreload 2

import collections
import ctypes
import itertools
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
sgo = ctypes.CDLL(s.SCORE_GUESS_OPT_SO)
from nerdle import Hint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Mini-Nerdle.
NUM_SLOTS = 6
SCORE_DB_FILE = "db/nerdle{}.db".format(NUM_SLOTS) 
solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DB_FILE)
d = solver_data.score_db
print(d.shape)

(206, 206)


## Solver

In [7]:
%timeit guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("4*3=12", initial_guess="56/7=8")

3.63 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Answer Generation

In [8]:
for num_slots in range(5, 9):
    print(num_slots)
    %time a = list(generator.all_answers(num_slots));
    print(len(a))

5
CPU times: user 2.7 ms, sys: 41 µs, total: 2.74 ms
Wall time: 2.78 ms
217
6
CPU times: user 49.1 ms, sys: 1.09 ms, total: 50.2 ms
Wall time: 49.9 ms
206
7
CPU times: user 635 ms, sys: 1.8 ms, total: 637 ms
Wall time: 638 ms
7561
8
CPU times: user 9.94 s, sys: 22.3 ms, total: 9.96 s
Wall time: 9.99 s
17723


## Score Database - Parallel Construction

In [10]:
answers = list(generator.all_answers(8))

In [11]:
n = 2000
a = answers[:n]
%time result = [nerdle._score_guess(guess, answer) for guess in a for answer in a]

TypeError: _score_guess() takes 1 positional argument but 2 were given

In [12]:
with multiprocessing.Pool(8) as pool:
    %time result1 = pool.starmap(nerdle._score_guess, itertools.product(a, repeat=2))

TypeError: _score_guess() takes 1 positional argument but 2 were given

In [13]:
# Slow.
# with multiprocessing.Pool(multiprocessing.cpu_count() // 2) as pool:
#     %time result2 = [pool.apply_async(nerdle._score_guess, t).get() for t in itertools.product(a, repeat=2)]
# Doesn't work yet.
# from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
# with ProcessPoolExecutor(max_workers=4) as pool:
#     %time result1 = pool.map(nerdle._score_guess, itertools.product(a, repeat=2))

In [14]:
from test_parallel import square

n = 10000000
with multiprocessing.Pool(processes=4) as pool:
    %time results = pool.map(square, itertools.repeat(3, n))

CPU times: user 519 ms, sys: 146 ms, total: 665 ms
Wall time: 966 ms


In [15]:
from test_parallel import process_one_arg

n = 30000000
print(n, n ** 0.5)
%time results = [process_one_arg("54/9=6") for _ in range(n)]
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    %time results = pool.map(process_one_arg, itertools.repeat("54/9=6", n))

30000000 5477.2255750516615
CPU times: user 15.8 s, sys: 274 ms, total: 16 s
Wall time: 16.1 s
CPU times: user 3.31 s, sys: 1.15 s, total: 4.46 s
Wall time: 5.31 s


In [18]:
# from test_parallel import process_args

# n = 2000
# a = answers[:n]
# %time result = [[nerdle._score_guess(guess, answer) for answer in a] for guess in a]

# with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
#     %time result2 = pool.map(process_args, itertools.product(a, repeat=2))
# result == result2
# np.array_equal(np.array(result2).reshape(n, n), np.array(result))

In [19]:
num_slots = 8

In [ ]:
# Parallel implementation.
%time solver_data = nerdle.create_solver_data(num_slots, "db/nerdle{}.db".format(num_slots), overwrite=True, min_parallel_n=len(answers) // 2)

In [ ]:
# Serial implementation.
%time solver_data = nerdle.create_solver_data(num_slots, "db/nerdle{}.db".format(num_slots), overwrite=True, min_parallel_n=2 * len(answers))

0 / 7561 (0.0%) completed
378 / 7561 (5.0%) completed
756 / 7561 (10.0%) completed
1134 / 7561 (15.0%) completed
1512 / 7561 (20.0%) completed
1890 / 7561 (25.0%) completed
2268 / 7561 (30.0%) completed
2646 / 7561 (35.0%) completed
3024 / 7561 (40.0%) completed
3402 / 7561 (45.0%) completed
3780 / 7561 (50.0%) completed
4158 / 7561 (55.0%) completed
